In [ ]:
import pandas as pd
features = pd.read_csv('../../0 - ModelData/FINAL:50MostImp+CombinationFeatures+Lagged.csv')
target = pd.read_csv('../../0 - ModelData/FINAL:Target---OutliersTreated.csv')

In [3]:
import numpy as np
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import pickle
from joblib import Parallel, delayed

# Ensure the features and target dataframes are correctly assigned
features = features.drop(columns=['Datum'])
target = target['PM10_Combined_Trend_Residual']

# Split the data into train, test, and validate sets
train_data, temp_data, train_target, temp_target = train_test_split(features, target, test_size=0.3, random_state=42)
test_data, validate_data, test_target, validate_target = train_test_split(temp_data, temp_target, test_size=0.3333, random_state=42)

# Add difference (growth/decrease) features
features_diff = features.diff().fillna(0)
features = pd.concat([features, features_diff.add_suffix('_diff')], axis=1)

# Add squared features
features_squared = features ** 2
features = pd.concat([features, features_squared.add_suffix('_squared')], axis=1)

# Standardize the features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Save the scaler using pickle
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

# Update the train, test, and validate sets with the scaled features
train_data, temp_data, train_target, temp_target = train_test_split(features_scaled, target, test_size=0.3, random_state=42)
test_data, validate_data, test_target, validate_target = train_test_split(temp_data, temp_target, test_size=0.3333, random_state=42)

# Define the parameter grid for GridSearchCV
param_grid = {
    'alpha': [0.0001, 0.001, 0.01, 0.1],
    'eta0': [0.001, 0.01, 0.1, 1],
    'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'],
    'penalty': ['l2', 'l1', 'elasticnet', None]
}

# Initialize the SGDRegressor
sgd = SGDRegressor(max_iter=1000, tol=1e-3)

# Initialize GridSearchCV
grid_search = GridSearchCV(sgd, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)

# Fit the model
grid_search.fit(train_data, train_target.values.ravel())

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Plot the progress bar
progress_bar = tqdm(total=100)
for i in range(100):
    progress_bar.update(1)
progress_bar.close()

# Print the best parameters and best score
print("Best Parameters:", best_params)
print("Best Score:", best_score)

# Evaluate the model on the test set
predictions = grid_search.predict(test_data)
predictions = np.maximum(predictions, 0)  # Ensure predictions are not lower than zero
mse = mean_squared_error(test_target, predictions)
print("Mean Squared Error on Test Set:", mse)

# Evaluate the model on the validation set
validate_predictions = grid_search.predict(validate_data)
validate_predictions = np.maximum(validate_predictions, 0)  # Ensure predictions are not lower than zero
validate_mse = mean_squared_error(validate_target, validate_predictions)
print("Mean Squared Error on Validation Set:", validate_mse)

# Function to train and evaluate model for each fold
def train_and_evaluate(fold):
    start_train_size = initial_train_size + fold * test_size
    end_train_size = start_train_size + test_size

    X_train_fold = train_data[:end_train_size]
    y_train_fold = train_target[:end_train_size]
    X_test_fold = train_data[end_train_size:end_train_size + test_size]
    y_test_fold = train_target[end_train_size:end_train_size + test_size]

    model = SGDRegressor(**best_params, max_iter=1000, tol=1e-3)
    model.fit(X_train_fold, y_train_fold)
    fold_predictions = model.predict(X_test_fold)
    fold_predictions = np.maximum(fold_predictions, 0)  # Ensure predictions are not lower than zero
    fold_mse = mean_squared_error(y_test_fold, fold_predictions)
    return fold, fold_mse

# Expanding CV Fold Sizes
initial_train_size = int(0.5 * len(train_data))
test_size = int(0.1 * len(train_data))
num_folds = (len(train_data) - initial_train_size) // test_size

# Use joblib to parallelize the computation
results = Parallel(n_jobs=-1)(delayed(train_and_evaluate)(fold) for fold in range(num_folds))

# Print the results for each fold
for fold, fold_mse in results:
    print(f"Fold {fold + 1}/{num_folds}, MSE: {fold_mse}")

Fitting 5 folds for each of 256 candidates, totalling 1280 fits
[CV] END alpha=0.0001, eta0=0.001, learning_rate=constant, penalty=l2; total time=   0.1s
[CV] END alpha=0.0001, eta0=0.001, learning_rate=constant, penalty=l2; total time=   0.2s
[CV] END alpha=0.0001, eta0=0.001, learning_rate=constant, penalty=l2; total time=   0.2s
[CV] END alpha=0.0001, eta0=0.001, learning_rate=constant, penalty=l2; total time=   0.2s
[CV] END alpha=0.0001, eta0=0.001, learning_rate=constant, penalty=l2; total time=   0.1s
[CV] END alpha=0.0001, eta0=0.001, learning_rate=constant, penalty=l1; total time=   0.2s
[CV] END alpha=0.0001, eta0=0.001, learning_rate=constant, penalty=l1; total time=   0.2s
[CV] END alpha=0.0001, eta0=0.001, learning_rate=constant, penalty=l1; total time=   0.2s
[CV] END alpha=0.0001, eta0=0.001, learning_rate=constant, penalty=elasticnet; total time=   0.2s
[CV] END alpha=0.0001, eta0=0.001, learning_rate=constant, penalty=elasticnet; total time=   0.2s
[CV] END alpha=0.000

/Users/johannessolibieda/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/Users/johannessolibieda/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/Users/johannessolibieda/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


[CV] END alpha=0.0001, eta0=0.01, learning_rate=adaptive, penalty=l2; total time=  16.1s
[CV] END alpha=0.0001, eta0=0.01, learning_rate=adaptive, penalty=l2; total time=  16.1s
[CV] END alpha=0.0001, eta0=0.01, learning_rate=adaptive, penalty=l2; total time=  16.2s


/Users/johannessolibieda/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/Users/johannessolibieda/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


[CV] END alpha=0.0001, eta0=0.01, learning_rate=adaptive, penalty=l2; total time=  16.3s
[CV] END alpha=0.0001, eta0=0.01, learning_rate=adaptive, penalty=l2; total time=  16.6s


...

/Users/johannessolibieda/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


[CV] END alpha=0.1, eta0=0.01, learning_rate=adaptive, penalty=None; total time=  18.6s


/Users/johannessolibieda/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


[CV] END alpha=0.1, eta0=0.01, learning_rate=adaptive, penalty=None; total time=  19.4s


KeyboardInterrupt: 

In [ ]:
# Define a more focused parameter grid for GridSearchCV based on the best parameters
refined_param_grid = {
    'alpha': [best_params['alpha'] * 0.1, best_params['alpha'], best_params['alpha'] * 10],
    'eta0': [best_params['eta0'] * 0.1, best_params['eta0'], best_params['eta0'] * 10],
    'learning_rate': [best_params['learning_rate']],
    'penalty': [best_params['penalty']]
}

# Initialize a new GridSearchCV with the refined parameter grid
refined_grid_search = GridSearchCV(sgd, refined_param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)

# Fit the model with the refined parameter grid
refined_grid_search.fit(train_data, train_target.values.ravel())

# Get the best parameters and best score from the refined search
refined_best_params = refined_grid_search.best_params_
refined_best_score = refined_grid_search.best_score_

# Print the refined best parameters and best score
print("Refined Best Parameters:", refined_best_params)
print("Refined Best Score:", refined_best_score)

# Evaluate the refined model on the test set
refined_predictions = refined_grid_search.predict(test_data)
refined_mse = mean_squared_error(test_target, refined_predictions)
print("Refined Mean Squared Error on Test Set:", refined_mse)

# Evaluate the refined model on the validation set
refined_validate_predictions = refined_grid_search.predict(validate_data)
refined_validate_mse = mean_squared_error(validate_target, refined_validate_predictions)
print("Refined Mean Squared Error on Validation Set:", refined_validate_mse)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] END alpha=0.005, eta0=0.0005, learning_rate=invscaling, penalty=l1; total time=   1.3s
[CV] END alpha=0.005, eta0=0.001, learning_rate=invscaling, penalty=l1; total time=   1.4s
[CV] END alpha=0.005, eta0=0.001, learning_rate=invscaling, penalty=l1; total time=   1.5s
[CV] END alpha=0.005, eta0=0.001, learning_rate=invscaling, penalty=l1; total time=   1.5s
[CV] END alpha=0.005, eta0=0.005, learning_rate=invscaling, penalty=l1; total time=   0.2s
[CV] END alpha=0.005, eta0=0.0005, learning_rate=invscaling, penalty=l1; total time=   1.7s
[CV] END alpha=0.005, eta0=0.005, learning_rate=invscaling, penalty=l1; total time=   0.2s
[CV] END alpha=0.005, eta0=0.0005, learning_rate=invscaling, penalty=l1; total time=   1.8s
[CV] END alpha=0.005, eta0=0.005, learning_rate=invscaling, penalty=l1; total time=   0.2s
[CV] END alpha=0.005, eta0=0.005, learning_rate=invscaling, penalty=l1; total time=   0.2s
[CV] END alpha=0.005, eta0=